In [1]:

#### 最初に必要なライブラリを読み込みます。
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit, QubitBra, measure_all, measure_all_oneshot,measure_partial, matrix_to_qubit
from sympy.physics.quantum.gate import H,X,Y,Z,S,T,CPHASE,CNOT,SWAP,UGate,CGateS,gate_simp,OneQubitGate
from sympy.physics.quantum.gate import IdentityGate as _I
from sympy.physics.quantum.qft import *
from sympy.physics.quantum.matrixcache import matrix_cache
matrix_cache.cache_matrix('Tdg',Matrix([[1, 0], [0, exp(-I*pi/4)]]))
matrix_cache.cache_matrix('Sdg',Matrix([[1, 0], [0, -I]]))
matrix_cache.cache_matrix('V',(1/2)*Matrix([[1+I, 1-I], [1-I, 1+I]]))
matrix_cache.cache_matrix('Vdg',(1/2)*Matrix([[1-I, 1+I], [1+I, 1-I]]))

CZ = CPHASE

class Tdg(OneQubitGate): # T^{\dagger} 演算
    gate_name = u'Tdg'
    gate_name_latex = u'T^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Tdg', format)

class Sdg(OneQubitGate): # S^{\dagger} 演算
    gate_name = u'Sdg'
    gate_name_latex = u'S^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Sdg', format)

class V(OneQubitGate): # √X 演算
    gate_name = u'V'
    gate_name_latex = u'V'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('V', format)

class Vdg(OneQubitGate): # √X ^{\dagger}演算
    gate_name = u'Vdg'
    gate_name_latex = u'V^{\dagger}'
    def get_target_matrix(self, format='sympy'):
        return matrix_cache.get_matrix('Vdg', format)

def CV(c,t):  return CGateS((c),V(t))
def CVdg(c,t):  return CGateS((c),Vdg(t))

def CCX(c1,c2,t):  return CGateS((c1,c2),X(t))
def Toffoli(c1,c2,t):  return CGateS((c1,c2),X(t))
def CCZ(c1,c2,t): return (H(t)*CCX(c1,c2,t)*H(t)) # CCZ演算子を定義します。
def hadamard(s,n):
    h = H(s)
    for i in range(s+1,n+s): h = H(i)*h
    return h

def disp1Q(u): print(u); display(represent(u,nqubits=1)); CircuitPlot(u,nqubits=1)
def disp2Q(u): print(u); display(represent(u,nqubits=2)); CircuitPlot(u,nqubits=2)

from sympy.printing.dot import dotprint
init_printing()

%matplotlib inline
import matplotlib.pyplot as plt
from sympy.physics.quantum.circuitplot import CircuitPlot,labeller, Mz,CreateOneQubitGate
alpha, beta, gamma, delta = Symbol(r'\alpha'), Symbol(r'\beta'), Symbol(r'\gamma'), Symbol(r'\delta'), 
psi, phi, theta, chi = Symbol(r'\psi'), Symbol(r'\phi'), Symbol(r'\theta'), Symbol(r'\chi') 
from qutip import *
import numpy as np


（副読本）
* ニールセン・チャン「量子コンピュータと量子通信」（ここでは N,C"QCQI" と書きます）
* arXivに公開されている論文

## 4.3 探索と最適化


### 4.3.1 概要

古典の「探索アルゴリズム」について

* 力まかせ探索

 しらみつぶしに探索する
 
* 近似解を求める「近似アルゴリズム」

（keyword）
 - ヒューリスティックス
 - 線形探索アルゴリズム
 - 二分技探索アルゴリズム（計算量 $O(\log{N})$）
 - 木構造を利用した「木探索」
 - グラフ構造を利用した「グラフ探索」
   - 幅優先探索
   - 深さ優先探索
  



### 4.3.2 グローバーの探索アルゴリズム

$ N $個のデータに対して、$ O(\sqrt{N}) $ 回のクエリで解を発見できる確率的なアルゴリズム

ここでは、$ N $ 個の要素から、$ M $ 個の解を探索する例。

3.5 節（振幅増幅サブルーチン）は、$ N $ 個の要素から、$ 1 $ 個を探索する

※コラムにあるように、グローバーのアルゴリズムは、「繰り返し回数の戦略」（何回繰り返すか）が密かに重要


https://github.com/kyamaz/openql-notes/blob/draft/docs/20210317/quantum_computing_3_1-3_5.ipynb

### 4.3.3 具体例：充足可能性問題

参考）[Groverのアルゴリズムを用いた充足可能性問題の解法](https://qiskit.org/textbook/ja/ch-applications/satisfiability-grover.html)


## 4.4 量子コンピュータと機械学習

### 4.4.1 機械学習

ここも古典な話

（keyword）

* 教師あり学習
* 教師なし学習
* 半教師あり学習
* 強化学習
* カーネルモデル
* ニューラルネットワーク
* 深層学習

* 目的関数（コスト関数）

 - 損失関数　（平均二乗誤差、ヒンジ損失関数、クロスエントロピー誤差関数　など）
 - 正則化項
 
* 過学習
* 正則化　（L2正則化、L1正則化、L0正則化）


→ [QML Club](https://qmlclub.connpass.com/) にて「[量子コンピュータによる機械学習](https://booklog.jp/item/1/4320124626)」が詳しい


（keyword）

* 最適化問題
* 連続最適化
* 離散最適化
* 凸計画
* 非凸計画
* 確率的勾配法（Stochastic Gradient Descent, SGD)

### 4.4.2 量子コンピュータで逆行列を高速に求める

**HHLアルゴリズム**

HHLアルゴリズムとは、ホロー(Aram W.Harrow)氏、ハシディム(Avinatan Hassidim)氏、ロイド(Seth Lloyd)氏らによって2009年に発表されたアルゴリズムで、量子位相推定を使用して効率的に逆行列計算を行う手法


■Quantum Algorithm for Linear Systems of Equations
(Aram W. Harrow, Avinatan Hassidim, and Seth Lloyd)
https://arxiv.org/pdf/0811.3171.pdf

参考）[HHL を利用して線形方程式系を解きQiskit で実装する](https://qiskit.org/textbook/ja/ch-applications/hhl_tutorial.html)


※「 i) $ b $ を振幅エンコーディングした $\displaystyle \lvert b \rangle = \sum _{i} b_{i} \lvert i \rangle $ を用意する」のに「2.7.2 QRAMを使った振幅エンコーディング」(p.47) を前提としています。


### 4.4.3 さまざまな量子機械学習アルゴリズム

位相推定、振幅増幅、グローバー探索、HHLアルゴリズム　を使う